In [4]:
import pandas as pd
import numpy as np

import psycopg2 as pg
import pandas.io.sql as psql

import matplotlib.pyplot as plt

%matplotlib inline

from algoliasearch import algoliasearch
from pymongo import MongoClient

import json
import time


In [3]:
2*229000/60/40


190

In [5]:
connection = pg.connect(dbname='brent', user='brent', host='localhost', password='brent')
cur = connection.cursor()

In [6]:
table_names = ['addresses','categories',
            'categories_products','category_features','clients',
               'companies','featured_items','invitations','order_items',
               'orders','popular_items','products','projects','projects_collaborators',
               'projects_products','proposal_product','proposals','saved_searches',
               'schema_version','showrooms','showrooms_vendors','spring_session',
               'spring_session_attributes','styles','user_addresses',
               'users','vendors','vendors_styles']

In [7]:
d = {}
for i in table_names:
    cur.execute('SELECT * FROM {}'.format(i))
    the_data = cur.fetchall()
    colnames = [desc[0] for desc in cur.description]

    d[i] = pd.DataFrame(the_data)
    if len(d[i].columns) == len(colnames):
        d[i].columns = colnames
    else:
        print i

categories_products


In [8]:
# d_dyn = {}
# c = 0
# for idx,i in enumerate(d['products']['dynamoid']):
#     if c == 1000:
#         break
#     elif "|" not in i:
#         d_dyn[d['products']['id'][idx]] = i
#         c+=1
#     else:
#         d_dyn[d['products']['id'][idx]] = i.split('|')[-1]
#         c+=1

# Importing libraries

In [9]:
# pymongo.version
import pprint
import pymongo
import pyspark
import pandas as pd
import numpy as np
import json


# Initialize Mongodb

In [10]:
cl = pymongo.MongoClient()

coll = cl.desinc_collection.products_db

# Initialize Algoliasearch

In [11]:

cl = algoliasearch.Client("88M6DNE4VR", 'a1d710a60be87b6f6688be3d3b13221c')
index = cl.init_index('di_raw_products_prod')


# Initialization of search algorithm
only to be run the first initialization.
After initialization, hashed out.

In [12]:
# Initialization of search algorithm
# items_max = {}
# items_max['max'] = 0
# items = {}

# Search / scrape algorithm

### Steps
Search using product dynamoids - unique ids used for lookup.

I could not successfully search for more than one item, max hit results was 20, regardless. Method: search for 1 at a time, which would be next on the list of 'dynamoids', the unique product ids.

Include a calculated break in the code for every 2+h of search.

Record "cursor" as a mutable dictionary value, the index number of the dynamoid list, as {'max' : index#}

Trial and error showed that a simple "items" {'dynamoid' : (hit result)} dictionary could be inputted one at a time "insert_one() method" into the mongodb.

Added a counter, for every 1k, to verify that the process was running.

If "try" was successful, then the "items" dictionary would be re-set to an empty dictionary.

Except was added, in the event that the dynamoid yielded a failed result. 2p run failed after 39k calls... and constant failure after that point continued due to "items = {}" being below insert, rather than being freshly initiated at the beginning of the try loop. Re-ran and discovered only specific fails in this grouping.

**in the future need to make a log for future troubleshooting**

1953 fails.

Typing those item ids into the website gets a result. For now, I need to move on with the data that I have available.


### Testing
Initially test for inputting 10 documents.

Then test for 1000.

Then run to complete groups of 100,000 documents.

### Performance
31k results could be gathered in 40min.

For 400k results, scraping takes 8.6h.





In [20]:
c = 0
for idx,i in enumerate(d['products']['dynamoid']):
    if c == 100000:
        break
    if idx > items_max['max']:
        try:
            items = {}
            query = i
            items[i] = index.search(query)['hits']
            coll.insert_one(items)
            items_max['max'] = idx
            c+=1
            if c%500 ==0:
                print c
            continue
        except:
            print 'fail: {}'.format(i), c
            c +=1

# len(items)
# items = {}

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
fail: d8vEAp6-Malibu-Wl-78.75 29039
fail: Q6gMO0q-Malibu-Wl-94.5 29285
29500
30000
fail: aavKQ4Q-Malibu-NO-78.75 30395
fail: Vl3oOGL-Malibu-NO-94.5 30396
fail: xo7jNqj-Malibu-WO-78.75 30398
fail: r7ajqEn-Malibu-WO-94.5 30399
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
fail: bYjqeED-CZ-01 LT. BLUE 41642
fail: rBMQg6a-CZ-01 LT. BLUE 41643
fail: 80zKEo5-CZ-01 LT. BLUE 41644
fail: 6XR1dEK-CZ-01 LT. BLUE 41645
fail: mR8d33d-AF-04 ANT. IVORY / RUST 41803
fail: DKwePPm-AF-04 ANT. IVORY / RUST 41804
fail: AogAbby-AF-04 ANT. IVORY / RUST 41805
fail: 2ZE7kko-AF-04 ANT. IVORY / RUST

fail: M06adLx-KH-05 BLUE / LT. BLUE 45595
fail: rBMQgRq-KH-05 BLUE / LT. BLUE 45596
fail: WYb9M5d-KH-05 BLUE / LT. BLUE 45597
fail: 47rxYJe-PJ-03 CHAMPAGNE / LT. GREY 45874
fail: ld2Nl1l-PJ-03 CHAMPAGNE / LT. GREY 45875
fail: P0oMZqE-PJ-03 CHAMPAGNE / LT. GREY 45876
fail: mR8dP9Z-PJ-03 CHAMPAGNE / LT. GREY 45877
fail: DKweZQE-PJ-03 CHAMPAGNE / LT. GREY 45878
fail: AogAZQG-PJ-03 CHAMPAGNE / LT. GREY 45879
fail: kdNez3o-PJ-03 CHAMPAGNE / LT. GREY 45880
fail: 2ZE7BNa-PJ-03 CHAMPAGNE / LT. GREY 45881
fail: ewm51l8-PJ-03 CHAMPAGNE / LT. GREY 45882
fail: N6a0Z3A-PJ-03 CHAMPAGNE / LT. GREY 45883
fail: dq5PgbJ-PJ-03 CHAMPAGNE / LT. GREY 45884
fail: 47rxYJa-PJ-03 CHAMPAGNE / LT. GREY 45885
fail: EzLBZmK-PJ-03 CHAMPAGNE / LT. GREY 45886
fail: R4Jrwb9-PJ-03 SILVER / LT. GREY 45887
fail: oRdEm1y-PJ-03 SILVER / LT. GREY 45888
fail: QMzbZ3b-PJ-03 SILVER / LT. GREY 45889
fail: YGJ7zV6-PJ-03 SILVER / LT. GREY 45890
fail: aYyJzdL-PJ-03 SILVER / LT. GREY 45891
fail: 07J9EnA-PJ-03 SILVER / LT. GREY 45892

In [ ]:
# cursor = coll.find({})
# for document in cursor:
#     print(document)

In [21]:
coll.count()

400092

In [22]:
items_max['max']

401906